In [4]:
import os
import nibabel as nib

t2flair_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/T2flair-study"
t1w_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/T1w-study"
roi_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/roi"

t2flair_nii_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/T2flair-study-nii"
t1w_nii_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/T1w-study-nii"
roi_nii_path = "/home/mariopasc/Python/Datasets/ds-epilepsy/roi-nii"

im_train_path = "/home/mariopasc/Python/Datasets/YolovV8-epilepsy/images/train"
im_val_path = "/home/mariopasc/Python/Datasets/YolovV8-epilepsy/images/val"
label_train_path = "/home/mariopasc/Python/Datasets/YolovV8-epilepsy/labels/train"
label_train_val = "/home/mariopasc/Python/Datasets/YolovV8-epilepsy/labels/val"

In [8]:
def convert_gz_nii(niigz_file, save_path, patient_id):
    # Verificar si el archivo .nii.gz existe
    if not os.path.isfile(niigz_file):
        print("Error: Specified .nii.gz file doesnt existe.")
        return

    # Cargar el archivo .nii.gz
    img = nib.load(niigz_file)
    data = img.get_fdata()
    # Obtener el número de rodajas
    num_slices = data.shape[2]

    # Crear el directorio de destino si no existe
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # Iterar sobre todas las rodajas y guardarlas como imágenes .nii
    for i in range(num_slices):
        # Obtener una sola rodaja
        slice_data = data[:, :, i]
        # Crear el nombre de la imagen
        image_name = f"{patient_id}-{i}.nii"
        # Guardar la imagen .nii
        nib.save(nib.Nifti1Image(slice_data, img.affine), os.path.join(save_path, image_name))

    print("Process finished with exit code 0")

def study_to_nii(study_path, save_path):
    if not os.path.exists(study_path):
        print("Input file does not exist")
        return
    
    # Por cada paciente dentro del estudio
    for patient_id in os.listdir(study_path):
        # Comprobar si el elemento es una carpeta
        patient_folder = os.path.join(study_path, patient_id)
        if not os.path.isdir(patient_folder):
            print(f"Patient {patient_folder} not found")
            continue

        # Extraemos el nombre de su archivo nii.gz
        niigz_files = [path for path in os.listdir(patient_folder) if path.endswith(".nii.gz")]
        if not niigz_files:
            print(f"No .nii.gz file found for patient {patient_id}")
            continue

        # Asumimos que solo hay un archivo .nii.gz por paciente
        niigz_file = os.path.join(patient_folder, niigz_files[0])
        convert_gz_nii(niigz_file=niigz_file, save_path=save_path, patient_id=patient_id)

study_to_nii(t2flair_path, t2flair_nii_path)

"""
patient = "sub-00001"
convert_nii_to_nii_slices(niigz_file= os.path.join(t2flair_path, patient, "sub-00001_acq-T2sel_FLAIR.nii.gz"),
                          save_path=  t2flair_nii_path, 
                          patient_id= patient)
"""



Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finished with exit code 0
Process finish

'\npatient = "sub-00001"\nconvert_nii_to_nii_slices(niigz_file= os.path.join(t2flair_path, patient, "sub-00001_acq-T2sel_FLAIR.nii.gz"),\n                          save_path=  t2flair_nii_path, \n                          patient_id= patient)\n'

ERROR (1): Parece que ha habido un problema con el nombre, hay pacientes que sí tienen roi pero que el nombre de su fichero T2 contiene solo la palabra FLAIR
SOLUCIÓN (1): Se ha solucionado al cambiar una línea en el ejecutable organizar_estudios.sh: 
```find "$folder/anat" -type f -name "*FLAIR*" -exec mv -t "$destination_t2flair_dir/$folder_name" {} +``` 
